<a href="https://colab.research.google.com/github/dohyeonkim2526/MyData-Contest/blob/master/My_Data_%EA%B0%80%ED%98%B9%EC%A1%B0%EA%B1%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [30]:
import os
import pandas as pd
import numpy as np

dt_dir='/content/gdrive/My Drive' 
dt_name='데이터-가혹.xlsx'
dt_dir=os.path.join(dt_dir, dt_name) #희승의 'wb'

In [31]:
#승이 기준

#from xlrd import oepn_workbook
#wb=open_workbook('데이터경로/data.xlsx')
#data=pd.read_excel(wb)

In [32]:
data=pd.read_excel(dt_dir) #희승의 'data'

In [33]:
print(data.dtypes)

주행 기록계 값                        float64
기록계 값 차이 (전일 기준) (km)           float64
시간당 기어 수                          int64
위도                              float64
경도                              float64
타이어 압력                          float64
가속(150-200km/h)                   int64
데이터 기준 일자                datetime64[ns]
데이터 기준 시간                        object
dtype: object


In [34]:
data['데이터 기준 일자']=pd.to_datetime(data['데이터 기준 일자'], format='%Y%m%d')

In [35]:
data

,주행 기록계 값,기록계 값 차이 (전일 기준) (km),시간당 기어 수,위도,경도,타이어 압력,가속(150-200km/h),데이터 기준 일자,데이터 기준 시간
0,57858.9,40.0,2,37.555179,126.970738,35.3,0,2020-08-31,23:50:00
1,57858.9,40.0,0,37.555179,126.970738,35.3,0,2020-09-01,00:00:00
2,57858.9,40.0,0,37.555179,126.970738,35.3,0,2020-09-01,00:10:00
3,57858.9,40.0,0,37.555179,126.970738,35.3,0,2020-09-01,00:20:00
4,57858.9,40.0,0,37.555179,126.970738,35.3,0,2020-09-01,00:30:00
...,...,...,...,...,...,...,...,...,...
141,57903.3,40.0,0,37.556635,126.923633,35.2,0,2020-09-01,23:20:00
142,57903.3,40.0,0,37.556635,126.923633,35.2,0,2020-09-01,23:30:00
143,57903.3,40.0,0,37.556635,126.923633,35.2,0,2020-09-01,23:40:00
144,57903.3,40.0,0,37.556635,126.923633,35.2,0,2020-09-01,23:50:00


In [36]:
date_list=data['데이터 기준 일자'].unique()

In [37]:
dist=[]
total_dist=[]

def find_odometer(dt):
  find=dt['주행 기록계 값']
  find=round(find, 3)
  total_dist.append(find)

In [38]:
def find_daily_distance(dt):
  find=dt['기록계 값 차이 (전일 기준) (km)']
  find=round(find,3)
  dist.append(find)

In [39]:
def div_data(dt): # div_data(data_list)

  for i in range(len(dt)):
    date=dt[i]
    n_data=data[data['데이터 기준 일자']==date]
    n_dist=n_data.iloc[-1]
    find_odometer(n_dist) 
    find_daily_distance(n_dist)


In [40]:
div_data(date_list) # total_dist, dist 생성

In [41]:
def with_n_data(dl): # with_n_data(date_list)
  harsh=0

  for i in range(len(dl)):
    sum=0
    date_dist=dist[i]
    date=dl[i]   
    n_data=data[data['데이터 기준 일자']==date]
    sum=n_data['가속(150-200km/h)'].sum()

    if sum >= (0.2*date_dist): # 일일 주행거리 기준 0.2배 이상 가속주행 확인
      harsh += 1
  
  return harsh

In [42]:
def pressure(dt): # pressure(data)
  MAX_Pres=44
  NORM_Pres=32
  
  harsh=0
  count=0
  pres=0

  first_pres=dt['타이어 압력'].iloc[0]

  for i in range(1, dt.shape[0]):
    pres=dt['타이어 압력'].iloc[i]
    if pres is not first_pres:
      count += 1

  if (count/len(date_list)) >= 0.3:
    harsh += 1
  
  return harsh

In [43]:
 def driven_distance(dt): # 20km 이하로 주 3-4회 이상 주행하면 가혹조건 +1 // driven_distance(dist)

   AVG_DIST=20
   NUM_DIST=3
   
   harsh=0
   avg_dist=np.mean(dt)
   num_dist=len(dt)
   count=0

   for i in range(len(dt)):
     check=dt[i]
     if check <= AVG_DIST:
       count+=1

   if count>= NUM_DIST:
     harsh=harsh+1
    
   return harsh

In [49]:
def Harsh_condition(db):
  h1=with_n_data(date_list)
  h2=pressure(db)
  h3=driven_distance(dist)
  harsh=h1+h2+h3
  
  if harsh >= 2:
    state='가혹 조건'
  
  else:
    state='통상 조건'

  return state

state=Harsh_condition(data)

In [50]:
print(state)

가혹 조건
